In [2]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('stopwords')

df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')

[nltk_data] Downloading package stopwords to /home/tomas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
def ver_info_a_filtrar(df, col):
    cant_publicaciones_columna_no_nulo = df[~df[col].isnull()].shape[0]
    print('La cantidad de publicaciones con ' + col +' no nulo es:', cant_publicaciones_columna_no_nulo)
    print('La cantidad total de publicaciones es:', df.shape[0])

In [4]:
def generar_palabras_no_queridas(arr=None):
    '''Retorna un set con las palabras a excluir, arr, es un campo opcional para excluir tambien 
    las palabras en ese arreglo'''
    stop_words = set(stopwords.words('spanish'))
    puntuacion = [',', '.', '!', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '-', '_', '\'', '\"', 
                  '(', ')', '[', ']', '{', '}', 'i', 'p', 'br', '<', '>', 'col.', '\\', '/', '$', 'm2', 
                  '..', '...', 'ii', '#', ';', ':', 'col']
    for x in puntuacion:
        stop_words.add(x)
    if arr:
        for x in arr:
            stop_words.add(x)
    return stop_words

In [5]:
def generar_palabras_mas_frecuentes(df, col, n):
    '''Esta funcion recibe un df, y genera la n palabras mas usadas la columna pasada por parametro, 
    retorna un arreglo ordenado de mas usadas a menos usadas'''
    reviews = df[col].str.cat(sep=' ')

    # Splitea los textos en palabras
    tokens = word_tokenize(reviews)

    # Fuera palabras no necesarias
    stop_words = generar_palabras_no_queridas()
    tokens = [w for w in tokens if not w in stop_words]

    vocabulary = set(tokens)
    
    # Calcula y ordena por frecuencia
    frequency_dist = nltk.FreqDist(tokens)
    mas_usadas = sorted(frequency_dist,key=frequency_dist.__getitem__, reverse=True)[0:n]
    return mas_usadas

In [6]:
def feature_cantidad_mas_frecuentes(df, col, n):
    '''Retorna el df pasado por parametro con la columna nueva "cant_palabras_mas_frecuentes", donde dice
    el numero de palabras mas frecuentes presentes en la columna pasada por parametro. n es la cantidad 
    palabras mas frecuentes a usar. Por ejemplo: `feature_cantidad_mas_frecuentes(df, 'titulo', 50)` genera 
    columna nueva `cant_palabras_mas_frecuentes_titulo`, donde se indica cuantas palabras frecuentes aparecen
    en el titulo, siendo las palabras mas frecuentes, la n palabras mas usadas entre todas las columnas 
    pasadas por parametro'''
    
    ver_info_a_filtrar(df, col)
    
    arr_mas_frecuentes = generar_palabras_mas_frecuentes(df, col, n)
    set_mas_frecuentes = set(arr_mas_frecuentes)
    
    def contador_mas_frecuentes(texto):
        contador = 0
        palabras_del_df = word_tokenize(texto)
        for palabra in palabras_del_df:
            if palabra in set_mas_frecuentes:
                contador = contador + 1
        return contador
    
    df['cant_palabras_mas_frecuentes_' + col] = df[~df[col].isnull()][col].apply(contador_mas_frecuentes)
    df['cant_palabras_mas_frecuentes_' + col] = df['cant_palabras_mas_frecuentes_' + col].fillna(0)
    
    return df

In [ ]:
def ejemplo():
    # Ejemplo de uso de las palabras mas frecuentes
    df_palabras = feature_cantidad_mas_frecuentes(df_train, 'titulo', 200)
    df_palabras[['id', 'titulo', 'cant_palabras_mas_frecuentes_titulo']].head()

La cantidad de publicaciones con titulo no nulo es: 234613
La cantidad total de publicaciones es: 240000


In [ ]:
def generar_palabras_menos_frecuentes(df, col, n):
    '''Esta funcion recibe un df, y genera la n palabras menos usadas la columna pasada por parametro, 
    retorna un arreglo ordenado de menos usadas a menos usadas'''
    reviews = df[col].str.cat(sep=' ')

    # Splitea los textos en palabras
    tokens = word_tokenize(reviews)

    # Fuera palabras no necesarias
    stop_words = generar_palabras_no_queridas()
    tokens = [w for w in tokens if not w in stop_words]

    vocabulary = set(tokens)
    
    # Calcula y ordena por frecuencia
    frequency_dist = nltk.FreqDist(tokens)
    menos_usadas = sorted(frequency_dist, key=frequency_dist.__getitem__, reverse=False)[0:n]
    return menos_usadas

In [ ]:
def feature_cantidad_menos_frecuentes(df, col, n):
    '''Retorna el df pasado por parametro con la columna nueva "cant_palabras_menos_frecuentes", donde dice
    el numero de palabras menos frecuentes presentes en la columna pasada por parametro. n es la cantidad 
    palabras menos frecuentes a usar. Por ejemplo: `cant_palabras_menos_frecuentes(df, 'titulo', 50)` genera 
    columna nueva `cant_palabras_menos_frecuentes_titulo`, donde se indica cuantas palabras menos frecuentes aparecen
    en el titulo, siendo las palabras menos frecuentes, la n palabras menos usadas entre todas las columnas 
    pasadas por parametro'''
    
    ver_info_a_filtrar(df, col)
    
    arr_menos_frecuentes = generar_palabras_menos_frecuentes(df, col, n)
    set_menos_frecuentes = set(arr_menos_frecuentes)
    
    def contador_menos_frecuentes(texto):
        contador = 0
        palabras_del_df = word_tokenize(texto)
        for palabra in palabras_del_df:
            if palabra in set_menos_frecuentes:
                contador = contador + 1
        return contador
    
    df['cant_palabras_menos_frecuentes_' + col] = df[~df[col].isnull()][col].apply(contador_menos_frecuentes)
    df['cant_palabras_menos_frecuentes_' + col] = df['cant_palabras_menos_frecuentes_' + col].fillna(0)
    
    return df

In [ ]:
def ejemplo():
    # Ejemplo de uso de las palabras menos frecuentes
    df_palabras = feature_cantidad_menos_frecuentes(df_train, 'titulo', 200)
    df_palabras[['id', 'titulo', 'cant_palabras_menos_frecuentes_titulo']].head()